In [ ]:
import nbimporter
import dataDownloader as dd

dd.downloadAndConvertToCSV()

In [15]:
import glob
import csv
import random

def getPreviousMatches(matches,x,teamVenue,predictionMatches):
    previousMatches = []
    for i in range(x-1,0,-1):
        if(len(previousMatches) == predictionMatches):
            break
        if(matches[x][teamVenue]==matches[i]['HomeTeam'] or matches[x][teamVenue]==matches[i]['AwayTeam']):
            previousMatches.append(matches[i])
    return previousMatches

def generateLearningSet(pastMatchesTaken, verbose = True):
    #Read files
    filesPaths = glob.glob("../data/*.csv")
    x_data = []
    y_data = []
    for dataFile in filesPaths: 
        if(verbose == True):
            print(f"Reading file {dataFile}")
        with open(dataFile) as csvfile:
            reader = csv.DictReader(csvfile)
            #print(f"reading file {dataFile} with {sum(1 for line in reader)} entries")
            matches = []
            try:
                for row in reader:
                    matches.append(row)
                for x in range(len(matches)):
                    homePreviousMatches = getPreviousMatches(matches, x, 'HomeTeam', pastMatchesTaken)
                    awayPreviousMatches = getPreviousMatches(matches, x,'AwayTeam', pastMatchesTaken)
                    temp = []
                    if len(homePreviousMatches) == pastMatchesTaken and len(awayPreviousMatches) == pastMatchesTaken:
                        #print(f"{x}: {matches[x]['HomeTeam']} vs {matches[x]['AwayTeam']}")

                        for hm in homePreviousMatches:
                            # temp = []
                            if matches[x]['HomeTeam'] == hm['HomeTeam']:
                                temp.append(int(float(hm['FTHG']))-int(float(hm['FTAG'])))
                                temp.append(int(float(hm['HS'])))
                                temp.append(int(float(hm['AS'])))
                                temp.append(int(float(hm['HST'])))
                                temp.append(int(float(hm['AST'])))
                                temp.append(int(float(hm['HC'])))
                                temp.append(int(float(hm['AC'])))
                            else:
                                temp.append(int(float(hm['FTAG']))-int(float(hm['FTHG'])))
                                temp.append(int(float(hm['AS'])))
                                temp.append(int(float(hm['HS'])))
                                temp.append(int(float(hm['AST'])))
                                temp.append(int(float(hm['HST'])))
                                temp.append(int(float(hm['AC'])))
                                temp.append(int(float(hm['HC'])))
                            #matchLearningData.append(temp)

                        for am in homePreviousMatches:
                            # temp = []
                            if matches[x]['AwayTeam'] == am['HomeTeam']:
                                temp.append(int(float(am['FTHG']))-int(float(am['FTAG'])))
                                temp.append(int(float(am['HS'])))
                                temp.append(int(float(am['AS'])))
                                temp.append(int(float(am['HST'])))
                                temp.append(int(float(am['AST'])))
                                temp.append(int(float(am['HC'])))
                                temp.append(int(float(am['AC'])))
                            else:
                                temp.append(int(float(am['FTAG']))-int(float(am['FTHG'])))
                                temp.append(int(float(am['AS'])))
                                temp.append(int(float(am['HS'])))
                                temp.append(int(float(am['AST'])))
                                temp.append(int(float(am['HST'])))
                                temp.append(int(float(am['AC'])))
                                temp.append(int(float(am['HC'])))
                            #matchLearningData.append(temp)
                            

                        x_data.append(temp)
                        if(int(float(matches[x]['FTHG'])) > int(float(matches[x]['FTAG']))):
                            y_data.append(1)
                        elif (int(float(matches[x]['FTAG'])) > int(float(matches[x]['FTHG']))):
                            y_data.append(-1)
                        else:
                            y_data.append(0)
            except Exception as e:
                if(verbose == True):
                    print(f"Problem with league {dataFile} : {e}")
                pass
                    
    return x_data, y_data

def createTrainAndTestSets(x_data, y_data, trainToOverallRatio):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(len(x_data)):
        if(random.random() < trainToOverallRatio):
            x_train.append(x_data[i])
            y_train.append(y_data[i])
        else:
            x_test.append(x_data[i])
            y_test.append(y_data[i])
    return x_train, y_train, x_test, y_test

In [20]:
x_data, y_data = generateLearningSet(10)
x_train, y_train, x_test, y_test = createTrainAndTestSets(x_data, y_data, 0.7)

print(f"overall matches count: {len(x_data)}")
print(f"train matches count: {len(x_train)}   {len(x_train)/len(x_data)}% of overall")
print(f"test matches count: {len(x_test)}")
print(x_train[0])
print(y_train[0])

Reading file ../data\B1-1314.csv
Problem with league ../data\B1-1314.csv : 'AS'
Reading file ../data\B1-1415.csv
Problem with league ../data\B1-1415.csv : 'AS'
Reading file ../data\B1-1516.csv
Problem with league ../data\B1-1516.csv : 'AS'
Reading file ../data\B1-1617.csv
Problem with league ../data\B1-1617.csv : 'AS'
Reading file ../data\B1-1718.csv
Reading file ../data\B1-1819.csv
Reading file ../data\B1-1920.csv
Reading file ../data\D1-1314.csv
Reading file ../data\D1-1415.csv
Reading file ../data\D1-1516.csv
Reading file ../data\D1-1617.csv
Reading file ../data\D1-1718.csv
Reading file ../data\D1-1819.csv
Reading file ../data\D1-1920.csv
Reading file ../data\D2-1314.csv
Problem with league ../data\D2-1314.csv : 'AS'
Reading file ../data\D2-1415.csv
Problem with league ../data\D2-1415.csv : 'AS'
Reading file ../data\D2-1516.csv
Problem with league ../data\D2-1516.csv : 'AS'
Reading file ../data\D2-1617.csv
Problem with league ../data\D2-1617.csv : 'AS'
Reading file ../data\D2-1718.c

In [25]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

def fit(x_train, y_train):
    clf.fit(x_train,y_train)
    # print(clf.class_count_)

def predict(x_test):
    return clf.predict(x_test)

def score(X,y):
    return clf.score(X,y)

In [26]:
fit(x_train, y_train)
predictions = predict(x_test)
print(score(x_test,y_test))
# print(predictions)
# print(y_test)

0.42150943396226415


In [18]:
import numpy
pastMatchesCountRange = range(1,10)
results = []

for x in pastMatchesCountRange:
    results.append([])
for i in range(3):
    print(f"Pass {i}")
    for x in pastMatchesCountRange:
        x_data, y_data = generateLearningSet(x, False)
        x_train, y_train, x_test, y_test = createTrainAndTestSets(x_data, y_data, 0.7)
        fit(x_train, y_train)
        predictions = predict(x_test)
        results[x-1].append(score(x_test,y_test))

print(results)

# print(results)

Pass 0
Pass 1
Pass 2
[[0.42457186838560707, 0.41175348215989316, 0.42491952281764817], [0.42638642194592463, 0.4276971671108479, 0.41650333202665624], [0.4198634263908415, 0.4222402104624102, 0.4198940073379535], [0.4216160576428204, 0.42351609916675237, 0.42705955592446565], [0.42818603647350817, 0.41859963318588844, 0.421272941934099], [0.4178501201660476, 0.4199416123961374, 0.414181577203446], [0.4183168316831683, 0.42105860972285325, 0.4204299852121488], [0.421461716937355, 0.4249208768022506, 0.42432369402985076], [0.4290318581876782, 0.42722778246714094, 0.4230165438956648]]


In [19]:
averages =[]
for x in range(len(results)):
    sum = 0
    for c in range(len(results[x])):
        sum += results[x][c]
    averages.append(sum/len(results[x]))
print(averages)

[0.4204149577877161, 0.4235289736944763, 0.42066588139706845, 0.42406390424467943, 0.4226862038644985, 0.41732443658854373, 0.4199351422060567, 0.4235687625898188, 0.4264253948501613]


In [49]:
# Custom predictions method
print(clf.classes_)

predictProba = clf.predict_proba(x_test)

customPredictions = []
customResults = []
for x in range(len(x_test)):
    if predictProba[x][0] > 0.9999999:
        customPredictions.append(clf.classes_[0])
    elif predictProba[x][1] > 0.9999999:
        customPredictions.append(clf.classes_[1])
    elif predictProba[x][2] > 0.9999999:
        customPredictions.append(clf.classes_[2])
    else:
        continue
    customResults.append(y_test[x])

correctPredictions = 0
for i in range(len(customPredictions)):
    if customPredictions[i] == customResults[i]:
        correctPredictions += 1

print(f"custom predictions: {correctPredictions/len(customPredictions)}") 
print(f"correct : {correctPredictions} negative : {len(customPredictions)-correctPredictions} thrown-away: {len(x_test)-len(customPredictions)} total : {len(x_test)}")

[-1  0  1]
custom predictions: 0.76
correct : 19 negative : 6 thrown-away: 7925 total : 7950


In [34]:
print(clf.predict_proba(x_test))

[[0.69333899 0.27792782 0.02873319]
 [0.56230107 0.30841345 0.12928548]
 [0.68853452 0.24650518 0.06496031]
 ...
 [0.30087708 0.4708303  0.22829262]
 [0.75481642 0.22897977 0.01620381]
 [0.05435381 0.1890429  0.7566033 ]]
